# MOFA results analysis in the colon for CD patients

### Imports

In [ ]:
import copy
import os
import random
import shutil

import matplotlib.pyplot as plt
import mofax as mfx
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.stats import chi2_contingency, f_oneway
from statsmodels.stats.multitest import multipletests
from sparc_multiomics.constants import (
    CCF_METADATA_CATEGORICAL,
    CCF_METADATA_CONTINUOUS,
    RANDOM_SEED,
)
from sparc_multiomics.MOFA_toolset import (
    grab_top_features_by_MOFA_weights,
    readable_features_naming,
    reverse_features_naming,
    select_top_factors,
)
from sparc_multiomics.pathway_analysis import (
    pathway_analysis,
)
from sparc_multiomics.plotting import (
    cleaner_barplot,
    custom_boxplot,
    two_groups_overlap_heatmap,
    custom_clustermap,
)

random.seed(RANDOM_SEED)
project_name = "uc_colon"

### Load the MOFA model

Loading the MOFA model, the relative weights of each feature towards the model, perform short preparations changes and merge with the metadata

In [ ]:
mofa_model = mfx.mofa_model(f"mofa_inflammation_{project_name}.h5")
weights_df = mofa_model.get_weights(df=True)
weights_df["feature"] = weights_df.index
tagged_weights_df = weights_df.merge(
    mofa_model.get_features(), on="feature", how="left"
)
tagged_weights_df.set_index("feature", inplace=True)
metadata = pd.read_parquet("collapsed_metadata.parquet")
metadata = metadata.rename(columns={"endo_category": "Disease Severity"})
factors_df = mofa_model.get_factors(df=True)
raw_factors_names = [col for col in factors_df if col.startswith("Factor")]
factors = [
    f"Factor {current_factor.replace('Factor','')}"
    for current_factor in raw_factors_names
]
renaming_factors_dict = dict(zip(raw_factors_names, factors))
tagged_weights_df = tagged_weights_df.rename(columns=renaming_factors_dict)
factors_df = factors_df.rename(columns=renaming_factors_dict)

number_of_factors = len(factors_df.columns)
factors_columns = list(factors_df.columns)
factors_r2 = mofa_model.get_r2()
factors_r2["Factor"] = factors_r2["Factor"].replace(renaming_factors_dict)

Plot the R2 of each factor towards the groups.

In [ ]:
mfx.plot_r2(
    mofa_model,
    factors=list(range(number_of_factors)),
    cmap="Blues",
    vmax=1,
    vmin=0.0,
)

Identify the top factors, per their R2 measured contribution.

In [ ]:
top_factors, factor_views = select_top_factors(
    factors_r2,
    verbose=True,
    r2_thresholds={"genomics": 0.1, "proteomics": 0.25, "transcriptomics": 0.25},
    return_mode="overlap",
    top_per_omics=5,
)

Select Factor 1 from the two top Factors identified.

In [ ]:
top_factors
top_factors = ["Factor 1"]

### Disease severity and Factor 1

Identifying whether disease severity has a pattern relating to Factor 1.

In [ ]:
grouping_variable = "Disease Severity"
factors_df["patient_id"] = [int(x.split("-")[0][1:]) for x in factors_df.index]
factors_df["interval_id"] = ["_".join(x.split("-")[1:]) for x in factors_df.index]

metadata["patient_id"] = metadata["patient_id"].astype(int)
factors_df["patient_id"] = factors_df["patient_id"].astype(int)
factors_with_metadata = factors_df.merge(
    metadata,
    on=["patient_id", "interval_id"],
    how="left",
)
custom_clustermap(
    factors_with_metadata,
    categories_to_display=[
        "Disease Severity",
        "batch",
    ],
    columns_to_cluster=factors_columns,
    save_path=f"figures/{project_name}_clustermap_all_factors.png",
    figsize=(8, 10),
)

Remove patients in remission as these can fall in any of the other categories, profile wise, at this point.

In [ ]:
endo_cat_subset = factors_with_metadata[
    factors_with_metadata[grouping_variable] != "remission"
]

Display the correlation between Factor 1 and disease severity.

In [ ]:
integer_index = {"remission": 0, "mild": 1, "moderate": 2, "severe": 3}
factors_with_metadata = factors_with_metadata.dropna(subset=[grouping_variable])
factors_with_metadata["index_dis_sev"] = factors_with_metadata[grouping_variable].map(
    integer_index
)
endo_category_order = list(integer_index.keys())
factors_with_metadata = factors_with_metadata.sort_values(
    "index_dis_sev", ascending=True
)

factors_with_metadata["abs Factor 1"] = factors_with_metadata["Factor 1"].abs()
endo_cat_subset = factors_with_metadata[
    factors_with_metadata[grouping_variable] != "remission"
]

r2_endo_cat_factor_1 = (
    endo_cat_subset["Factor 1"].abs().corr(endo_cat_subset["index_dis_sev"])
)
print(f"R2 between disease severity and Factor 1: {round(r2_endo_cat_factor_1,2)}")
custom_boxplot(
    endo_cat_subset,
    x_variable=grouping_variable,
    y_variable="abs Factor 1",
    grouping_variable=grouping_variable,
    save_tag="abs_Factor_1",
    circle_size=3,
    fig_size=(6, 6),
    title=f"Disease severity vs Factor 1 (R2: {round(r2_endo_cat_factor_1,2)})",
    title_fontsize=15,
    pairs_to_annotate=[
        ("severe", "moderate"),
        ("severe", "mild"),
        ("moderate", "mild"),
    ],
    legend_spacing=-0.34,
    prettify_y_axis=False,
    add_stats=True,
    grouped=False,
    skip_sorting=True,
)

### Loop back to the original data

In an effort to locate biomarkers able to characterize the samples, we import them once again and perform statistical testing to see their significance against the clusters attained.

In [ ]:
factors_with_metadata.to_parquet(f"MOFA_results_{project_name}.parquet")
transcriptomics = pd.read_parquet("transcriptomics_batch_corrected.parquet")
proteomics = pd.read_parquet("proteomics_processed.parquet")
genomics = pd.read_parquet("genomics_annotated.parquet")


transcriptomics_columns = list(
    transcriptomics.drop(columns=["original_batch", "sample_id"]).columns
)
transcriptomics_scaler = StandardScaler()
scaled_transcriptomics = transcriptomics_scaler.fit_transform(
    transcriptomics.drop(columns=["original_batch", "sample_id"])
)
scaled_transcriptomics = pd.DataFrame(
    scaled_transcriptomics, columns=transcriptomics_columns
)

processed_transcriptomics = scaled_transcriptomics[transcriptomics_columns]
transcriptomics_columns = list(processed_transcriptomics.columns)
processed_transcriptomics[["original_batch", "sample_id"]] = transcriptomics[
    ["original_batch", "sample_id"]
]

proteomics_columns = list(proteomics.drop(columns=["sample_id"]).columns)
proteomics_scaler = StandardScaler()
scaled_proteomics = proteomics_scaler.fit_transform(
    proteomics.drop(columns=["sample_id"])
)
scaled_proteomics = pd.DataFrame(scaled_proteomics, columns=proteomics_columns)
scaled_proteomics["sample_id"] = proteomics["sample_id"]

genomics_columns = list(genomics.drop(columns=["sample_id"]).columns)
MOFA_proteomics = pd.merge(
    factors_with_metadata, scaled_proteomics, left_on="proteomics", right_on="sample_id"
).drop(columns=["sample_id"])

MOFA_proteomics_transcriptomics = pd.merge(
    MOFA_proteomics,
    processed_transcriptomics,
    left_on="transcriptomics",
    right_on="sample_id",
).drop(columns=["sample_id"])
MOFA_proteomics_transcriptomics_genomics = pd.merge(
    MOFA_proteomics_transcriptomics,
    genomics,
    left_on="genomics_array",
    right_on="sample_id",
).drop(columns=["sample_id"])

Pick 150 top features for each of the omics types.

In [ ]:
top_sizes = {
    "genomics": 150,
    "proteomics": 150,
    "transcriptomics": 150,
}

Count the genes that appear the most on the top features, from the absolute weights towards factor 3.

In [ ]:
top_features_columns, top_features = {}, {}
for usable_omics in ["genomics", "proteomics", "transcriptomics"]:
    top_features_columns[usable_omics] = reverse_features_naming(
        grab_top_features_by_MOFA_weights(
            tagged_weights_df,
            top_factors,
            current_view=usable_omics,
            top_n=top_sizes[usable_omics],
        ),
        split_parameters={
            "split_by": "-",
            "split_when": [
                "Inflammation",
                "Cardiometabolic",
                "Oncology",
                "Neurology",
                "chr",
            ],
        },
    )
    top_features[usable_omics] = readable_features_naming(
        top_features_columns[usable_omics],
        split_parameters={
            "split_by": "-",
            "split_when": [
                "Inflammation",
                "Cardiometabolic",
                "Oncology",
                "Neurology",
                "chr",
            ],
        },
    )

Count the genes that appear the most on the top features, from the absolute weights towards factor 3.

In [ ]:
genomics_counts_dictionary = {}
for current_gene in [x.split("-")[0] for x in top_features["genomics"]]:
    if current_gene == "":
        continue
    if current_gene in genomics_counts_dictionary:
        genomics_counts_dictionary[current_gene] += 1
    else:
        genomics_counts_dictionary[current_gene] = 1
genomics_counts_df = pd.DataFrame(
    genomics_counts_dictionary.items(), columns=["gene", "count"]
)
genomics_counts_df = genomics_counts_df.sort_values("count", ascending=False)
genomics_counts_df.to_csv(f"results/{project_name}_genomics_counts.csv", index=False)

cleaner_barplot(
    genomics_counts_df,
    input_x="count",
    input_y="gene",
    save_info=f"{project_name}_genomics_counts",
    title="Genomics top features counts",
)

Enrich each of the omics types with features from the remaining omics types, when possible.

In [ ]:
intersecting_features = {
    "genomics": top_features_columns["genomics"],
    "proteomics": top_features_columns["proteomics"],
    "transcriptomics": top_features_columns["transcriptomics"],
}
full_features_lists = {
    "genomics": genomics_columns,
    "proteomics": proteomics_columns,
    "transcriptomics": transcriptomics_columns,
}
omics_list = ["genomics", "proteomics", "transcriptomics"]
for usable_omic in omics_list:
    for current_feature in top_features[usable_omic]:
        if current_feature == "":
            continue
        nested_omics = omics_list.copy()
        nested_omics.remove(usable_omic)
        for nested_omic in nested_omics:
            for nested_feature in full_features_lists[nested_omic]:
                if current_feature in nested_feature:
                    intersecting_features[nested_omic].append(nested_feature)

intersecting_features = {
    key: list(set(value)) for key, value in intersecting_features.items()
}
proteomics_usable_columns = [
    x
    for x in intersecting_features["proteomics"]
    if x in MOFA_proteomics_transcriptomics_genomics.columns
]
proteomics_corrected_columns = [
    x.replace("-", "_")
    for x in intersecting_features["proteomics"]
    if x not in MOFA_proteomics_transcriptomics_genomics.columns
]
intersecting_features["proteomics"] = (
    proteomics_usable_columns + proteomics_corrected_columns
)

Test whether the features are statistically significant regarding the disease severity as a group.

In [ ]:
## Intersecting omics
custom_palette = sns.color_palette(["#16EB96", "#F4B183"])
types_columns = {
    "genomics": intersecting_features["genomics"],
    "proteomics": intersecting_features["proteomics"],
    "transcriptomics": intersecting_features["transcriptomics"],
    "metadata_continuous": CCF_METADATA_CONTINUOUS,
    "metadata_categorical": CCF_METADATA_CATEGORICAL,
}
target_group = "severe"
variables_types = {
    "genomics": "categorical",
    "proteomics": "continuous",
    "transcriptomics": "continuous",
    "metadata_continuous": "continuous",
    "metadata_categorical": "categorical",
}

valid_features = {}
type_columns_combinations = {}
for current_type in types_columns.keys():
    if current_type not in type_columns_combinations.keys():
        type_columns_combinations[current_type] = []
    for current_column in types_columns[current_type]:
        type_columns_combinations[current_type].append(current_column)

for current_type in tqdm(type_columns_combinations):
    variable_type = variables_types[current_type]
    if current_type not in valid_features.keys():
        valid_features[current_type] = []
    unique_groups = MOFA_proteomics_transcriptomics_genomics[grouping_variable].unique()
    groups_list = []
    for current_group in MOFA_proteomics_transcriptomics_genomics[
        grouping_variable
    ].unique():
        current_subset = MOFA_proteomics_transcriptomics_genomics[
            MOFA_proteomics_transcriptomics_genomics[grouping_variable] == current_group
        ]
        if current_subset.shape[0] < 2:
            continue
        groups_list.append(current_subset[type_columns_combinations[current_type]])

    if variable_type == "categorical":

        p_values = []
        for target_column in type_columns_combinations[current_type]:
            try:
                contingency_table = pd.crosstab(
                    MOFA_proteomics_transcriptomics_genomics[grouping_variable].astype(
                        "category"
                    ),
                    MOFA_proteomics_transcriptomics_genomics[target_column],
                )
                chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            except:
                p_value = 1
            p_values.append(p_value)

    if variable_type == "continuous":
        f, p_values = f_oneway(*groups_list)
    p_values_df = pd.DataFrame(
        p_values, index=type_columns_combinations[current_type], columns=["p_values"]
    )
    p_values_df["p_values"] = (
        p_values_df["p_values"].fillna(1).replace([0, np.inf, -np.inf], 1)
    )
    corrected_p_values = multipletests(
        p_values_df["p_values"].tolist(), method="fdr_bh", alpha=0.05
    )[1]
    p_values_df["corrected_p_values"] = corrected_p_values
    p_values_df = p_values_df[p_values_df["corrected_p_values"] < 0.05]
    valid_features[current_type] = p_values_df.index.tolist()

Make a dictionary with all possible features for each omics type.

In [ ]:
joint_feature_dictionary = {}
for current_omics in valid_features.keys():
    if current_omics not in joint_feature_dictionary.keys():
        joint_feature_dictionary[current_omics] = valid_features[current_omics]
    else:
        joint_feature_dictionary[current_omics] = list(
            set(joint_feature_dictionary[current_omics])
            | set(valid_features[current_omics])
        )

### Significant features

Plot individual plots for each relevant feature against macroscopic appearance, boxplot for continuous variables and heatmap for categorical.

In [ ]:
custom_palette = sns.color_palette(["#16EB96", "#F4B183"])
if os.path.exists(f"figures/individual_analysis/{project_name}") == True:
    shutil.rmtree(f"figures/individual_analysis/{project_name}")
os.makedirs(f"figures/individual_analysis/{project_name}", exist_ok=True)


for current_omics in joint_feature_dictionary.keys():
    table_replicate = copy.deepcopy(MOFA_proteomics_transcriptomics_genomics)
    current_type = variables_types[current_omics]

    for current_column in joint_feature_dictionary[current_omics]:
        single_column_replicate = copy.deepcopy(
            MOFA_proteomics_transcriptomics_genomics[
                [
                    current_column,
                    grouping_variable,
                ]
            ]
        )
        single_column_replicate = single_column_replicate.dropna(
            subset=[current_column]
        )

        plt.clf()
        if current_type == "continuous":
            custom_boxplot(
                single_column_replicate,
                x_variable=grouping_variable,
                y_variable=current_column,
                grouping_variable=grouping_variable,
                save_tag=f"individual_analysis/{project_name}/{current_omics}",
                circle_size=3,
                fig_size=(6, 4),
                title=f"{current_column.split(' ')[0].split('_')[0].upper()}",
                pairs_to_annotate=[
                    ("severe", "mild"),
                    ("severe", "moderate"),
                    ("moderate", "mild"),
                    ("severe", "remission"),
                    ("moderate", "remission"),
                    ("mild", "remission"),
                ],
                title_fontsize=15,
                legend_title="",
                legend_spacing=-0.34,
                prettify_y_axis=False,
                add_stats=True,
                grouped=False,
                skip_sorting=True,
                order=endo_category_order,
            )

        elif current_type == "categorical":
            single_column_replicate[current_column] = single_column_replicate[
                current_column
            ].astype(str)
            if current_omics == "genomics":
                single_column_replicate = single_column_replicate[
                    single_column_replicate[current_column] != "3"
                ]
            two_groups_overlap_heatmap(
                single_column_replicate,
                column_1=grouping_variable,
                column_2=current_column,
                save_tag=f"/individual_analysis/{project_name}/{current_omics}",
                normalize_row_wise=True,
            )

### Pathway analysis

Perform pathway analysis on the top found genes for each omics type.

In [ ]:
pathway_results = {}

try:
    pathway_results["genomics"] = pathway_analysis(
        valid_features["genomics"],
        p_value=0.05,
        number_of_pathways=10,
        save_info=f"{project_name}_genomics",
        return_df=True,
        remove_pathway_source=True,
        split_parameters={"split_by": "_", "split_when": ["chr"]},
    )
except:
    pass

try:
    pathway_results["proteomics"] = pathway_analysis(
        valid_features["proteomics"],
        p_value=0.05,
        number_of_pathways=10,
        save_info=f"{project_name}_proteomics",
        return_df=True,
        remove_pathway_source=True,
        color="#F4B183",
        split_parameters={
            "split_by": "_",
            "split_when": [
                "Inflammation",
                "Cardiometabolic",
                "Oncology",
                "Neurology",
            ],
        },
    )
except:
    pass

try:
    pathway_results["transcriptomics"] = pathway_analysis(
        valid_features["transcriptomics"],
        p_value=0.05,
        number_of_pathways=10,
        save_info=f"{project_name}_transcriptomics",
        return_df=True,
        remove_pathway_source=True,
        split_parameters={"split_by": "_", "split_when": []},
    )
except:
    pass